In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Parkinsons_Disease"
cohort = "GSE49126"

# Input paths
in_trait_dir = "../../input/GEO/Parkinsons_Disease"
in_cohort_dir = "../../input/GEO/Parkinsons_Disease/GSE49126"

# Output paths
out_data_file = "../../output/preprocess/Parkinsons_Disease/GSE49126.csv"
out_gene_data_file = "../../output/preprocess/Parkinsons_Disease/gene_data/GSE49126.csv"
out_clinical_data_file = "../../output/preprocess/Parkinsons_Disease/clinical_data/GSE49126.csv"
json_path = "../../output/preprocess/Parkinsons_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptomic profiling of peripheral blood mononuclear cells from patients with Parkinson's disease and control subjects"
!Series_summary	"To get insight into systemic molecular events associated with Parkinson's disease (PD), an age-related neurodegenerative disorder, we compared gene expression patterns of peripheral blood mononuclear cells (PBMC) derived from elderly healhy controls and from PD patients."
!Series_overall_design	"Transcriptomic profiling of patients with Parkinson's disease and control subjects. RNA were extracted from peripheral mononuclear blood cells and were hybridized on 4x44k Agilent expression microarrays."
Sample Characteristics Dictionary:
{0: ['disease state: control', "disease state: Parkinson's disease"], 1: ['cell type: peripheral blood mononuclear cells']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Yes, the background information suggests this dataset contains gene expression data from Agilent microarrays
is_gene_available = True

# 2. Variables Availability and Data Type Conversion
# 2.1 Data Availability

# For Parkinson's Disease (trait)
# From sample characteristics, the first row (index 0) contains disease state (control vs. Parkinson's disease)
trait_row = 0 

# No age information available in the characteristics dictionary
age_row = None  

# No gender information available in the characteristics dictionary
gender_row = None  

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert trait value to binary: 1 for Parkinson's disease, 0 for control."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if "parkinson" in value.lower():
        return 1
    elif "control" in value.lower():
        return 0
    else:
        return None

def convert_age(value):
    """Convert age value to continuous numeric."""
    # Not used since age data is not available
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender value to binary: 0 for female, 1 for male."""
    # Not used since gender data is not available
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value = value.lower()
    if 'female' in value or 'f' in value:
        return 0
    elif 'male' in value or 'm' in value:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering of dataset usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # The sample characteristics dictionary only contains the information about available variables
    # We need to construct clinical_data DataFrame in a format compatible with geo_select_clinical_features
    
    # For this dataset, we need to create a DataFrame with the disease state information
    sample_characteristics = {0: ['disease state: control', "disease state: Parkinson's disease"]}
    
    # Create a DataFrame from the sample characteristics
    clinical_data = pd.DataFrame(sample_characteristics)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the resulting DataFrame
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{0: [0.0]}
Clinical data saved to ../../output/preprocess/Parkinsons_Disease/clinical_data/GSE49126.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyzing gene identifiers from the previous step output
# These identifiers appear to be numeric values rather than gene symbols
# Human gene symbols typically follow nomenclature patterns like BRCA1, TP53, etc.
# The numeric identifiers (12, 13, 14, etc.) likely need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['266', '266', '266', '266', '266'], 'ROW': [170.0, 168.0, 166.0, 164.0, 162.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'GO_ID': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan, nan, nan, nan], 'SPOT_ID.1': [nan, nan, nan, nan, nan], 'ORDER': [1.0, 2.0, 3.

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Analyze the gene expression data indices and gene annotation columns to determine mapping
# From the gene annotation preview, we can see that 'ID' could be the gene identifier and 'GENE_SYMBOL' looks like the column for gene symbols

# Get a more comprehensive view of the gene annotation data to confirm our column choices
print("First few rows of gene annotation with non-null gene symbols:")
non_null_gene_symbols = gene_annotation.dropna(subset=['GENE_SYMBOL']).head(5)
print(non_null_gene_symbols[['ID', 'GENE_SYMBOL']])

# 2. Extract the mapping between gene identifiers and gene symbols
# The 'ID' column in gene_annotation seems to match the index in gene_data
mapping_data = get_gene_mapping(gene_annotation, 'ID', 'GENE_SYMBOL')

print("Preview of mapping data:")
print(mapping_data.head())
print(f"Number of mappings: {len(mapping_data)}")

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_data)

print("Preview of gene expression data after mapping:")
print(f"Number of genes after mapping: {len(gene_data)}")
print(gene_data.head())

# Normalize gene symbols if needed
gene_data = normalize_gene_symbols_in_index(gene_data)

print("Preview of gene expression data after normalization:")
print(f"Number of genes after normalization: {len(gene_data)}")
print(gene_data.head())


First few rows of gene annotation with non-null gene symbols:
    ID   GENE_SYMBOL
11  12      APOBEC3B
13  14        ATP11B
14  15  LOC100132006
15  16        DNAJA1
17  18         EHMT2
Preview of mapping data:
    ID          Gene
11  12      APOBEC3B
13  14        ATP11B
14  15  LOC100132006
15  16        DNAJA1
17  18         EHMT2
Number of mappings: 32696


Preview of gene expression data after mapping:
Number of genes after mapping: 18379
        GSM1194062   GSM1194063   GSM1194064   GSM1194065   GSM1194066  \
Gene                                                                     
A1BG   1346.496358  1630.091552  2081.789469  1796.719663  2008.200069   
A1CF      4.538759     5.038548     3.803502     8.955622     3.817977   
A2BP1     9.909301     9.704206     7.470562     8.859836     7.333760   
A2LD1   669.066900   548.872900   403.105000   463.760900   319.830400   
A2M      42.052910    38.886210    23.065940    43.597190    26.161890   

        GSM1194067   GSM1194068   GSM1194069   GSM1194070   GSM1194071  ...  \
Gene                                                                    ...   
A1BG   1522.559963  1009.084895  1341.158950  2027.324354  2003.861906  ...   
A1CF      5.237851     3.216614     3.457880     4.237291     5.366957  ...   
A2BP1     9.685222    16.132344     6.926038     8.215888     9.822340  ...   
A2

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols from the already mapped gene expression data from Step 6
# Gene data was already normalized in Step 6
print(f"Gene expression data shape: {gene_data.shape}")
print("First 5 gene symbols:")
print(gene_data.index[:5])

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Re-extract clinical data correctly to ensure it matches with genetic data
# Re-obtain the original data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_df = get_background_and_clinical_data(matrix_file)

# Print sample identifiers from original clinical data to debug
print("Original clinical data columns (should contain sample IDs):")
print(clinical_df.columns[:5])  # Show first 5 sample IDs

# Print sample identifiers from genetic data to debug
print("Genetic data columns (sample IDs):")
print(gene_data.columns[:5])  # Show first 5 sample IDs

# Extract clinical features directly from the original clinical_df
# The trait information is in row 0 (disease state)
def convert_trait(value):
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if "parkinson" in value.lower():
        return 1
    elif "control" in value.lower():
        return 0
    else:
        return None

# Create clinical data with the trait information
trait_row = 0  # The disease state row
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_df,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=None,
    convert_age=None,
    gender_row=None,
    convert_gender=None
)

print("Selected clinical data shape:", selected_clinical_df.shape)
print("Selected clinical data preview (transposed):")
# Transpose to show samples as rows to make structure clearer
print(selected_clinical_df.T.head())

# Save the processed clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
# Both datasets should have samples as columns initially
# Check for matching sample IDs
clinical_samples = set(selected_clinical_df.columns)
genetic_samples = set(gene_data.columns)
common_samples = clinical_samples.intersection(genetic_samples)

print(f"Number of clinical samples: {len(clinical_samples)}")
print(f"Number of genetic samples: {len(genetic_samples)}")
print(f"Number of common samples: {len(common_samples)}")

if len(common_samples) > 0:
    # Filter both datasets to include only common samples
    clinical_data_filtered = selected_clinical_df[list(common_samples)]
    gene_data_filtered = gene_data[list(common_samples)]
    
    # Now link them using geo_link_clinical_genetic_data function
    linked_data = geo_link_clinical_genetic_data(clinical_data_filtered, gene_data_filtered)
    print(f"Linked data shape: {linked_data.shape}")
    
    # 4. Handle missing values
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Data shape after handling missing values: {linked_data.shape}")
    
    # 5. Determine if trait and demographic features are biased
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Data shape after removing biased features: {linked_data.shape}")
    print(f"Is the trait biased: {is_biased}")
    
    # 6. Validate and save cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note="Dataset contains gene expression data from blood samples of PD patients and controls."
    )
    
    # 7. Save linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset deemed not usable. Linked data was not saved.")
else:
    print("No common samples found between clinical and genetic data. Cannot link the datasets.")
    # Record that the dataset is not usable
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,  # No common samples means trait data isn't available for gene samples
        is_biased=True,  # Mark as biased since we can't use it
        df=pd.DataFrame(),  # Empty dataframe
        note="No common samples between clinical and genetic data."
    )
    print("Dataset deemed not usable. Linked data was not saved.")

Gene expression data shape: (17901, 50)
First 5 gene symbols:
Index(['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2'], dtype='object', name='Gene')


Normalized gene data saved to ../../output/preprocess/Parkinsons_Disease/gene_data/GSE49126.csv
Original clinical data columns (should contain sample IDs):
Index(['!Sample_geo_accession', 'GSM1194062', 'GSM1194063', 'GSM1194064',
       'GSM1194065'],
      dtype='object')
Genetic data columns (sample IDs):
Index(['GSM1194062', 'GSM1194063', 'GSM1194064', 'GSM1194065', 'GSM1194066'], dtype='object')
Selected clinical data shape: (1, 50)
Selected clinical data preview (transposed):
            Parkinsons_Disease
GSM1194062                 0.0
GSM1194063                 0.0
GSM1194064                 0.0
GSM1194065                 0.0
GSM1194066                 0.0
Clinical data saved to ../../output/preprocess/Parkinsons_Disease/clinical_data/GSE49126.csv
Number of clinical samples: 50
Number of genetic samples: 50
Number of common samples: 50
Linked data shape: (50, 17902)


Data shape after handling missing values: (50, 17902)
For the feature 'Parkinsons_Disease', the least common label is '0.0' with 20 occurrences. This represents 40.00% of the dataset.
The distribution of the feature 'Parkinsons_Disease' in this dataset is fine.

Data shape after removing biased features: (50, 17902)
Is the trait biased: False


Linked data saved to ../../output/preprocess/Parkinsons_Disease/GSE49126.csv
